In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
# 用於二元目標變數
from scipy.stats import pointbiserialr
from functools import reduce
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

In [3]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo, Dream_report
from module1 import predict_for_month, find_best_threshold, predicted_labels, predicted_accuracy

c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Dream_report = pd.read_sql(qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stock_info = pd.read_sql(stock_qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:112: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MacroInfo = pd.read_sql(macro_qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:124: UserWarning: pandas

In [4]:
len(Dream_report)

187652

##### MacroEcon
FX-Foriegn-exchange rate


CCI-台灣消費者信心指數

### 預計可使用的變數

In [5]:
# 名目、實質、時間
"""
券商預估、歷史營收增長
產業: 存貨量、產能利用率、訂單數量

MacroInfo
名目有效匯率指數–台灣
台灣消費者信心指數
台灣消費者信心指數–未來半年國內經濟景氣
台灣消費者信心指數–未來半年國內物價水準
美國消費者信心指數
經濟成長率(GDP)–全年
--
實質國內生產毛額(GDP)–台幣 -> 只更新到202009
實質GDP成長率(台灣) -> 只更新到202212
--

sysallun
貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均
臺灣製造業PMI - 新增出口訂單 - 基礎原物料
台灣外銷訂單金額–全球
消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數
美國核心PPI、美國核心PPI月增率
進口物價指數、出口物價指數年增率

sysallunTWESPF
外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)
景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)

sysTaiStaValMAC
景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)

其他
財報、營收
"""

'\n券商預估、歷史營收增長\n產業: 存貨量、產能利用率、訂單數量\n\nMacroInfo\n名目有效匯率指數–台灣\n台灣消費者信心指數\n台灣消費者信心指數–未來半年國內經濟景氣\n台灣消費者信心指數–未來半年國內物價水準\n美國消費者信心指數\n經濟成長率(GDP)–全年\n--\n實質國內生產毛額(GDP)–台幣 -> 只更新到202009\n實質GDP成長率(台灣) -> 只更新到202212\n--\n\nsysallun\n貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均\n臺灣製造業PMI - 新增出口訂單 - 基礎原物料\n台灣外銷訂單金額–全球\n消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數\n美國核心PPI、美國核心PPI月增率\n進口物價指數、出口物價指數年增率\n\nsysallunTWESPF\n外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)\n景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)\n\nsysTaiStaValMAC\n景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)\n\n其他\n財報、營收\n'

### *【訓練步驟：資料處理 → 分割 train、test dataset → model fit  → 計算出機率&閾值 → 計算準確率】

### 總經資料處理

In [6]:
# Improt data
# 名目有效匯率指數
NominalExchageRate = MacroInfo[MacroInfo['名稱']=='名目有效匯率指數–台灣'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(NominalExchageRate.head(3))

# 台灣消費者信心指數
TW_ICS = MacroInfo[MacroInfo['名稱']=='台灣消費者信心指數'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_ICS.tail(3))

# 消費者物價指數(CPI)年增率
TW_CPIGrowth = MacroInfo[MacroInfo['名稱']=='消費者物價指數(CPI)年增率'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_CPIGrowth.head(3))

# 經濟成長率(GDP)–全年
TW_GDPGrowth = MacroInfo[MacroInfo['名稱']=='經濟成長率(GDP)–單季'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_GDPGrowth.head(3))

,ddate,value,category
1573,202312,101.78,名目有效匯率指數–台灣
2802,202311,101.05,名目有效匯率指數–台灣
4052,202310,100.99,名目有效匯率指數–台灣


,ddate,value,category
259617,201403,80.96,台灣消費者信心指數
261932,201402,82.93,台灣消費者信心指數
264227,201401,80.83,台灣消費者信心指數


,ddate,value,category
820,202312,2.71,消費者物價指數(CPI)年增率
2062,202311,2.90,消費者物價指數(CPI)年增率
3312,202310,3.05,消費者物價指數(CPI)年增率


,ddate,value,category
0,202412,1.06,經濟成長率(GDP)–單季
27,202409,2.12,經濟成長率(GDP)–單季
42,202406,4.77,經濟成長率(GDP)–單季


In [7]:
MacroInfo.head()

,年月,代號,數值,名稱
0,202412,19600,1.06,經濟成長率(GDP)–單季
1,202412,19700,201644.00,國內生產毛額(GDP)–美元
2,202412,19710,6404219.00,國內生產毛額(GDP)–台幣
3,202412,19750,8567.00,平均每人國內生產毛額(GDP)–美元
4,202412,19760,272086.00,平均每人國內生產毛額(GDP)–台幣


In [8]:
"""
# 合計買賣超金額(百萬) = 上市櫃合計買進金額(百萬)-上市櫃合計賣出金額(百萬)、興櫃買賣超金額(百萬) 
TradingExceeds2 = JuridicalInfo2[['日期', '名稱','合計買賣超金額(百萬)','興櫃買賣超金額(百萬)']]
TradingExceeds2 = TradingExceeds2.fillna(0)

# Create English Column names
TradingExceeds2 = TradingExceeds2.rename({'日期':'ddate', '名稱':'classification', '合計買賣超金額(百萬)':'Trans_amount(mln)', '興櫃買賣超金額(百萬)':'Emerge_amount(mln)'}, axis='columns') 
#TradingExceeds2.head(10)
"""

"\n# 合計買賣超金額(百萬) = 上市櫃合計買進金額(百萬)-上市櫃合計賣出金額(百萬)、興櫃買賣超金額(百萬) \nTradingExceeds2 = JuridicalInfo2[['日期', '名稱','合計買賣超金額(百萬)','興櫃買賣超金額(百萬)']]\nTradingExceeds2 = TradingExceeds2.fillna(0)\n\n# Create English Column names\nTradingExceeds2 = TradingExceeds2.rename({'日期':'ddate', '名稱':'classification', '合計買賣超金額(百萬)':'Trans_amount(mln)', '興櫃買賣超金額(百萬)':'Emerge_amount(mln)'}, axis='columns') \n#TradingExceeds2.head(10)\n"

### 三大法人買賣超資料處理

In [9]:
# 上市櫃公司三大法人買賣超、持股比率增減(分個股)
JuridicalInfo = JuridicalInfo.rename({'日期':'ddate', '股票代號':'stockid', '三大法人月初以來買賣超金額(千)':'GrandTransAmount(K)', '三大法人持股比率增減':'ShareRatio_change', '三大法人近1日買賣超金額(千)':'DailyTransAmount(K)'}, axis=1)
#display(JuridicalInfo.head())

In [10]:
JuridicalInfo['date'] = pd.to_datetime(JuridicalInfo['ddate']) # make sure the date column is datetime type
JuridicalInfo['last_date_of_month'] = JuridicalInfo.groupby([JuridicalInfo['date'].dt.year, JuridicalInfo['date'].dt.month])['date'].transform(max)  # get the last day of the month
# display(JuridicalInfo[JuridicalInfo['ddate']=='20230424'])


C:\Users\user1\AppData\Local\Temp\ipykernel_18996\1795286498.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  JuridicalInfo['last_date_of_month'] = JuridicalInfo.groupby([JuridicalInfo['date'].dt.year, JuridicalInfo['date'].dt.month])['date'].transform(max)  # get the last day of the month


In [11]:
TW_NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
TW_NominalGDP = TW_NominalGDP.rename(columns={'time':'ddate'})
#print(TW_NominalGDP.head(3))
TW_NominalGDP['ddate'] = TW_NominalGDP['ddate'].astype(str)

In [12]:
# 將GDP季資料先合併
GDP_quarterly = pd.merge(TW_NominalGDP, TW_GDPGrowth, on='ddate', how='left')
GDP_quarterly.columns = ['ddate', 'NominalGDP', 'GDP_growth', 'category_GDPgrowth']
#GDP_quarterly.head(3)

In [13]:
# (月)總經資料
dfs = [NominalExchageRate, TW_ICS, TW_CPIGrowth]
MergedMacroData = reduce(lambda left, right: pd.merge(left, right, on='ddate', how='left'), dfs)

# Rename columns
MergedMacroData.columns = ['ddate', 'FX', 'category_FX', 'CCI',
                          'category_CCI', 'CPI_growth', 'category_CPIgrowth']

#display(MergedMacroData)
# FX-Foriegn-exchange rate
# CCI-台灣消費者信心指數

In [14]:
MergedMacroData = pd.merge(MergedMacroData, GDP_quarterly, on='ddate', how='left')

In [15]:
MergedMacroData['NominalGDP'] = MergedMacroData['NominalGDP'].ffill()
MergedMacroData['GDP_growth'] = MergedMacroData['GDP_growth'].ffill()
MergedMacroData['category_GDPgrowth'] = MergedMacroData['category_GDPgrowth'].ffill()

In [16]:
MergedMacroData.head(9)

,ddate,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202312,101.78,名目有效匯率指數–台灣,70.06,台灣消費者信心指數,2.71,消費者物價指數(CPI)年增率,NaN,NaN,NaN
1,202311,101.05,名目有效匯率指數–台灣,69.40,台灣消費者信心指數,2.90,消費者物價指數(CPI)年增率,NaN,NaN,NaN
2,202310,100.99,名目有效匯率指數–台灣,69.06,台灣消費者信心指數,3.05,消費者物價指數(CPI)年增率,NaN,NaN,NaN
3,202309,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202308,100.67,名目有效匯率指數–台灣,67.51,台灣消費者信心指數,2.53,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
5,202307,101.31,名目有效匯率指數–台灣,68.39,台灣消費者信心指數,1.88,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
6,202306,102.80,名目有效匯率指數–台灣,66.66,台灣消費者信心指數,1.75,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
7,202305,101.84,名目有效匯率指數–台灣,63.67,台灣消費者信心指數,2.02,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
8,202304,101.35,名目有效匯率指數–台灣,63.18,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季


### 單純以總經變數訓練模型，準確率約51% -TheWhole

In [17]:
# Rename the column
df = df.rename(columns={'direction':'real_direction'})

In [18]:
# 合併df以及總經資料
TheWhole = pd.merge(df[['ddate', 'stockid', 'real_direction']].astype({'ddate':'str'}), 
                    MergedMacroData, on='ddate', how='left')
#TheWhole[TheWhole['ddate']=='202302'].head(3)

In [19]:
#TheWhole.columns
TheWhole = TheWhole.dropna()
#TheWhole.head()

In [20]:
# Split data by TimeSeriesSplit
tscv_macro = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_macro = list(tscv_macro.split(TheWhole))

In [21]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_macro = splits_macro[-1][0]
test_idx_macro = splits_macro[-1][1]

In [22]:
TargetCol = ['real_direction']
VarialbleCol = ['FX', 'CCI', 'CPI_growth',
                'NominalGDP', 'GDP_growth']

# Prepare data for separating
y_macro = TheWhole[TargetCol]
X_macro = TheWhole[VarialbleCol]

In [23]:
# Extract test and train data from overall data
X_train_macro, X_test_macro = X_macro.iloc[train_idx_macro], X_macro.iloc[test_idx_macro]
y_train_macro, y_test_macro = y_macro.iloc[train_idx_macro], y_macro.iloc[test_idx_macro]

In [24]:
# Train model based on the training data
model_macro = LogisticRegression()
model_fit_macro = model_macro.fit(X_train_macro, y_train_macro)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
# Make predictions on the test data
# Calculate thresholds to determine the predicted results
# probability
# display the results(0/1)

In [26]:
# 預測結果為1的機率
prob_macro = predict_for_month(model_fit_macro, TheWhole, VarialbleCol)
threshold_macro = find_best_threshold(TheWhole['real_direction'], prob_macro)
#print(round(threshold_macro, 2)) #0.51

print(round(threshold_macro, 4))

0.5102


In [27]:
# 以閾值為界判斷預測值應為0或1
pred_macro = predicted_labels(prob_macro, threshold_macro)

In [28]:
# 計算總經數值正確性
predicted_accuracy(TheWhole['real_direction'], pred_macro) #0.51

0.51

### 總經指數+營收 訓練+預測 -TheWhole_rev

In [29]:
TheWhole['ddate'] = TheWhole['ddate'].astype(str)
df['ddate'] = df['ddate'].astype(str)

In [30]:
df = df.rename({'direction':'real_direction'}, axis='columns')
TheWhole_rev = pd.merge(df, TheWhole, on=('ddate', 'stockid', 'real_direction'), how='inner')
TheWhole_rev.head(5)

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,mond12,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,-2.67,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,-1.06,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,12.55,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,-1.38,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,-11.13,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季


In [31]:
Varialble_MacroRev = ['revd1', 'yoyd1',
                         'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
                         'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
                         'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
                         'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
                         'revd12', 'yoyd12', 'mond12', 'FX', 'CCI', 'CPI_growth',
                         'NominalGDP', 'GDP_growth']

Target_MacroRev = ['real_direction']

In [32]:
# Prepare data for separating
y_MacroRev = TheWhole_rev[Target_MacroRev]
X_MacroRev = TheWhole_rev[Varialble_MacroRev]

# Extract test and train data from overall data
X_train_MacroRev, X_test_MacroRev = X_MacroRev.iloc[train_idx_macro], X_MacroRev.iloc[test_idx_macro]
y_train_MacroRev, y_test_MacroRev = y_MacroRev.iloc[train_idx_macro], y_MacroRev.iloc[test_idx_macro]

In [33]:
# Train model based on the training data
model_MacroRev = LogisticRegression()
model_fit_MacroRev = model_MacroRev.fit(X_train_MacroRev, y_train_MacroRev)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
# 預測結果為1的機率
prob_MacroRev = predict_for_month(model_fit_MacroRev, TheWhole_rev, Varialble_MacroRev)
threshold_MacroRev = find_best_threshold(TheWhole_rev['real_direction'], prob_MacroRev) # 0.5092
print(threshold_MacroRev)
#print(round(threshold_MacroRev, 4))

# 以閾值為界判斷預測值應為0或1
pred_MacroRev = predicted_labels(prob_MacroRev, threshold_MacroRev)

0.5091613123764465


In [35]:
# 計算總經數值正確性
predicted_accuracy(TheWhole_rev['real_direction'], pred_MacroRev) # 0.62

0.62

In [36]:
# 分月份


### 以MLE判斷模型及變數是否有效

In [37]:
# 針對總經+12月營收判斷
# 使用 RFE 進行變數選擇
selector = RFE(model_MacroRev, n_features_to_select=20) #留下20 個 features
selector = selector.fit(X_MacroRev, y_MacroRev)

# Get the ranking of each feature
featre_ranking = selector.ranking_

featre_ranking
# 輸出選擇的特徵
selected_features = X_MacroRev.columns[selector.support_]
print(selected_features)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIO

Index(['revd1', 'yoyd1', 'mond1', 'yoyd2', 'yoyd3', 'mond3', 'yoyd4', 'mond4',
       'yoyd5', 'yoyd6', 'mond6', 'yoyd7', 'mond7', 'yoyd8', 'yoyd9', 'yoyd10',
       'mond11', 'revd12', 'yoyd12', 'mond12'],
      dtype='object')


In [38]:
selected_features = np.where(featre_ranking == 1)[0]

In [39]:
# 標準化
scaler = StandardScaler()
X_train_MacroRev = scaler.fit_transform(X_train_MacroRev)
X_test_MacroRev = scaler.transform(X_test_MacroRev)

model = LogisticRegression()
model.fit(X_train_MacroRev, y_train_MacroRev)

coeffients = model.coef_[0]
feature_importance = pd.DataFrame({'Feature': Varialble_MacroRev, 'Importance': np.abs(coeffients)})
feature_importance = feature_importance.sort_values('Importance', ascending=False)
#feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))
display(feature_importance[:20])

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Feature,Importance
0,revd1,2.141860
33,revd12,1.481950
3,revd2,0.445347
15,revd6,0.383429
6,revd3,0.243646
18,revd7,0.217388
30,revd11,0.213559
12,revd5,0.156035
27,revd10,0.145844
2,mond1,0.075035


### 以重要度最高的20個變數訓練模型

In [40]:
#(feature_importance['Feature'][:20][0])
Varialble_import20 = list(feature_importance['Feature'][:20].astype(str))

In [41]:
Varialble_import20

['revd1',
 'revd12',
 'revd2',
 'revd6',
 'revd3',
 'revd7',
 'revd11',
 'revd5',
 'revd10',
 'mond1',
 'revd4',
 'CPI_growth',
 'CCI',
 'GDP_growth',
 'revd9',
 'yoyd1',
 'revd8',
 'NominalGDP',
 'FX',
 'mond7']

In [42]:
# Prepare data for separating
y_import20 = TheWhole_rev[Target_MacroRev]
X_import20 = TheWhole_rev[Varialble_import20]

# Separate data into test and train set
X_train_import20, X_test_import20 = X_import20.iloc[train_idx_macro], X_import20.iloc[test_idx_macro]
y_train_import20, y_test_import20 = y_import20.iloc[train_idx_macro], y_import20.iloc[test_idx_macro]

# Train model based on the training data
model_import20 = LogisticRegression()
model_fit_import20 = model_import20.fit(X_train_import20, y_train_import20)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [43]:
# 預測結果為1的機率
prob_import20 = predict_for_month(model_fit_import20, X_import20, Varialble_import20)
threshold_import20 = find_best_threshold(y_import20, prob_import20) # 0.5092
#print(threshold_import20)
#print(round(threshold_import20, 4)) 

# 以閾值為界判斷預測值應為0或1
pred_import20 = predicted_labels(prob_import20, threshold_import20)
#print(pred_import20)

# 計算總經數值正確性
predicted_accuracy(y_import20, pred_import20) # 0.62

0.62

###### Conclusion: because the threshold is almost the same as the unselect variable ones, the accuracy is the same.

### 以前20重要的變數+三大法人+其他總經變數訓練 -TheWhole_rev_jur1

#### Data Cleaning

In [44]:
Dream_report['ddate'] = Dream_report['ddate'].astype(str)
Dream_report['stockid'] = Dream_report['stockid'].astype(int)

In [45]:
Dream_report.columns

Index(['ddate', 'stockid', 'direction', 'ActualDirect'], dtype='object')

In [46]:
# 將TheWhole_rev結合夢幻報表的預測方向(direction)及實際變動方向(ActualDirect)
TheWhole_rev = TheWhole_rev.merge(Dream_report, how='inner', on=['ddate', 'stockid'])
TheWhole_rev

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth,direction,ActualDirect
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,1
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,1
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155167,201601,9951,0,340061.0,-4.64,19.32,356608.0,31.13,7.53,331642.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,1,0
155168,201601,9955,0,110743.0,-47.49,-63.31,210914.0,-16.54,86.76,112934.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0
155169,201601,9958,0,217788.0,-25.39,8.63,291900.0,19.28,-15.76,346505.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0
155170,201601,9960,0,33867.0,-20.50,-14.70,42599.0,-5.87,3.90,41000.0,...,名目有效匯率指數–台灣,80.89,台灣消費者信心指數,0.80,消費者物價指數(CPI)年增率,4318253.0,-0.09,經濟成長率(GDP)–單季,0,0


In [47]:
# Check for duplicate rows in TheWhole_rev
duplicates = TheWhole_rev.duplicated(subset=['ddate', 'stockid'], keep=False)
print("Number of duplicate rows in TheWhole_rev:", duplicates.sum())

Number of duplicate rows in TheWhole_rev: 0


In [48]:
if 'ActualDirect' in TheWhole_rev:
    print("Merge successful: 'ActualDirect' is in the merged dataframe.")
else:
    print("Merge unsuccessful: 'ActualDirect' is NOT in the merged dataframe.")

Merge successful: 'ActualDirect' is in the merged dataframe.


In [49]:
TheWhole_rev = TheWhole_rev.drop('real_direction', axis=1)

In [50]:
print(len(TheWhole_rev.columns))
print(len(TheWhole_rev))

52
155172


In [51]:
# 將 Thewhole_rev 加上三大法人資料
len(JuridicalInfo) #6809784

6824584

In [52]:
#JuridicalInfo.tail(3)
#JuridicalInfo[JuridicalInfo['ddate'] == '20231222']

In [53]:
#JuridicalInfo_trans = JuridicalInfo
JuridicalInfo.columns
# 創建一個新的欄位 'year_month' 來儲存每個日期的年份和月份
JuridicalInfo['year_month'] = JuridicalInfo['date'].dt.to_period('M')

# 計算每個年份和月份的 'ShareRatio_change' 總和
JuridicalInfo['monthly_ShareRatio_change'] = JuridicalInfo.groupby(['year_month', 'stockid'])['ShareRatio_change'].transform('sum')

# 只在 'last_date_of_month' 等於 'date' 的那一行顯示 'monthly_ShareRatio_change'
JuridicalInfo.loc[JuridicalInfo['last_date_of_month'] != JuridicalInfo['date'], 'monthly_ShareRatio_change'] = None

JuridicalInfo

,ddate,stockid,GrandTransAmount(K),ShareRatio_change,DailyTransAmount(K),date,last_date_of_month,year_month,monthly_ShareRatio_change
0,20240205,1101,66439.0,-0.11,29703.0,2024-02-05,2024-02-05,2024-02,-0.05
1,20240205,1101B,NaN,0.00,NaN,2024-02-05,2024-02-05,2024-02,0.00
2,20240205,1102,-78780.0,-0.10,-82800.0,2024-02-05,2024-02-05,2024-02,-0.10
3,20240205,1103,1039.0,-0.01,-667.0,2024-02-05,2024-02-05,2024-02,0.01
4,20240205,1104,4105.0,-0.02,-3074.0,2024-02-05,2024-02-05,2024-02,-0.03
...,...,...,...,...,...,...,...,...,...
6824579,20060102,9949,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN
6824580,20060102,9950,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN
6824581,20060102,9951,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN
6824582,20060102,9955,0.0,0.00,0.0,2006-01-02,2006-01-25,2006-01,NaN


In [54]:
# Create a boolean mask where 'date' is the same as 'last_date_of_month'
mask2 = (JuridicalInfo['date'] == JuridicalInfo['last_date_of_month'])

# Use the mask to index the DataFrame
JuridicalInfo_last_day = JuridicalInfo[mask2]
JuridicalInfo_last_day = JuridicalInfo_last_day.rename({'ddate': 'ddate_with_date'})
JuridicalInfo_last_day['ddate'] = JuridicalInfo_last_day['year_month'].dt.strftime('%Y%m')

#JuridicalInfo_last_day.columns
JuridicalInfo_last_day = JuridicalInfo_last_day.drop(columns=['ShareRatio_change','DailyTransAmount(K)', 'date','last_date_of_month','year_month'], axis=1)

In [55]:
JuridicalInfo_last_day.head()
#JuridicalInfo_last_day[JuridicalInfo_last_day['ddate'] == '202311']

,ddate,stockid,GrandTransAmount(K),monthly_ShareRatio_change
0,202402,1101,66439.0,-0.05
1,202402,1101B,NaN,0.00
2,202402,1102,-78780.0,-0.10
3,202402,1103,1039.0,0.01
4,202402,1104,4105.0,-0.03


In [56]:
type(JuridicalInfo_last_day['stockid'][0]) #str
type(TheWhole_rev['stockid'][0]) # int

JuridicalInfo_last_day['stockid'] = pd.to_numeric(JuridicalInfo_last_day['stockid'], errors='coerce')
JuridicalInfo_last_day = JuridicalInfo_last_day.dropna(subset=['stockid'])
JuridicalInfo_last_day['stockid'] = JuridicalInfo_last_day['stockid'].astype(int)
#JuridicalInfo_last_day.head()
#len(JuridicalInfo_last_day)

In [57]:
TheWhole_rev_jur = pd.merge(TheWhole_rev, JuridicalInfo_last_day, how='inner', on=('ddate', 'stockid'))
#TheWhole_rev_jur

In [58]:
categories_current_sales =  ['貨幣總額年增率(M1A)日平均', '貨幣總額年增率(M2)日平均', 
                                                        '台灣外銷訂單金額–全球', '台灣外銷訂單金額–大陸及香港',
                                                        '台灣外銷訂單金額–資訊與通信產品', '台灣外銷訂單年增率',
                                                        '進口物價指數年增率']

categories_prosperity = ['台灣景氣領先指標', '台灣景氣同時指標', '台灣景氣落後指標']

In [59]:
MacroInfo[(MacroInfo['名稱'] == '貨幣總額年增率(M2)日平均') & (MacroInfo['年月'] == '201601')][['年月', '數值']]

,年月,數值
202318,201601,5.63


In [60]:
# 對於每一個類別，提取相應的數值並合併到 'TheWhole_rev_jur' DataFrame
TheWhole_rev_jur1 = TheWhole_rev_jur.copy()

for categories in categories_current_sales:
    # 從 MacroInfo 選出該名稱的數據
    temp_df = MacroInfo[MacroInfo['名稱'] == categories][['年月', '數值']].rename(columns={'年月':'ddate','數值':categories})
    TheWhole_rev_jur1 = pd.merge(TheWhole_rev_jur1, temp_df, how= 'inner', on='ddate')

In [61]:
len(TheWhole_rev_jur1) #151239

151239

In [62]:
for categories in categories_prosperity:
    # 從 MacroInfo 選出該名稱的數據
    temp_df = MacroInfo[MacroInfo['名稱'] == categories][['年月', '數值']].rename(columns={'年月':'ddate','數值':categories})
    TheWhole_rev_jur1 = pd.merge(TheWhole_rev_jur1, temp_df, how='inner', on='ddate')

In [63]:
#TheWhole_rev_jur1.tail()

In [64]:
#MacroInfo[(MacroInfo['名稱'] == '台灣景氣領先指標') & (MacroInfo['年月'] == '201601')][['年月', '數值']]

#### prepare training & testing dataset

In [65]:
# Target variabls (實際變動方向)
y_Important20Jur = TheWhole_rev_jur['ActualDirect']

# Create a new variable list
VarialbleCol_Important20Jur = Varialble_import20 + categories_current_sales + categories_prosperity

# Extract specific columns
X_Important20Jur = TheWhole_rev_jur1[VarialbleCol_Important20Jur]

In [66]:
# Split data by TimeSeriesSplit
tscv_MacroJur = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_MacroJur = list(tscv_MacroJur.split(TheWhole_rev_jur1))

In [67]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_MacroJur = splits_MacroJur[-1][0]
test_idx_MacroJur = splits_MacroJur[-1][1]

In [68]:
# Extract test and train data from TheWhole_rev_jur1 data
X_train_MacroJur, X_test_MacroJur = X_Important20Jur.iloc[train_idx_MacroJur], X_Important20Jur.iloc[test_idx_MacroJur]
y_train_MacroJur, y_test_MacroJur = y_Important20Jur.iloc[train_idx_MacroJur], y_Important20Jur.iloc[test_idx_MacroJur]

#### Data Training (Logistic Regression)

In [69]:
# Train model based on the training data
model_MacroJur = LogisticRegression()
model_fit_MacroJur = model_MacroJur.fit(X_train_MacroJur, y_train_MacroJur)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Probability Pediction

In [70]:
# 預測結果為1的機率
prob_MacroJur = predict_for_month(model_fit_MacroJur, TheWhole_rev_jur1, VarialbleCol_Important20Jur)
threshold_MacroJur = find_best_threshold(TheWhole_rev_jur['ActualDirect'], prob_MacroJur)
print(round(threshold_MacroJur, 2)) #0.499 -> 0.5

#print(round(threshold_macro, 4)) # 0.5102

0.49


In [71]:
# 以閾值為界判斷預測值應為0或1
pred_MacroJur = predicted_labels(prob_MacroJur, threshold_MacroJur)

In [72]:
# 計算總經數值正確性
predicted_accuracy(TheWhole_rev_jur['ActualDirect'], pred_MacroJur) #0.59

0.58

### Correct Process (improtant 20 variables + other macro variables)

#### Data Cleaning

In [73]:
#* data 清理-切割-標準化-特徵篩選-數據訓練-精準度檢查.....

#### Data Splitting

In [74]:
# Split data by TimeSeriesSplit
tscv_MacroJur = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_MacroJur = list(tscv_MacroJur.split(TheWhole_rev_jur1))

#### 使用所有分割組合進行交叉驗證

In [75]:
# 初始化模型
model = LogisticRegression()

# 存儲每一個分割的精準度
accuracies = []

# 存儲精確度最高的分割
best_accuracy = 0
best_split = None

# 對每一個分割進行訓練和評估
for train_index, test_index in tscv_MacroJur.split(X_Important20Jur):
    X_train, X_test = X_Important20Jur.iloc[train_index], X_Important20Jur.iloc[test_index]
    y_train, y_test = y_Important20Jur[train_index], y_Important20Jur[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

    # 如果這個分割的精確度比目前最好的分割的精確度還要高，則更新最好的分割
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_split = (train_index, test_index)

# Best Accuracy
print("Best Accuracy", best_accuracy)
print("Best split", best_split)

# 計算平均精準度
#average_accuracy = sum(accuracies) / len(accuracies)
#print("Average accuracy: ", average_accuracy)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Best Accuracy 0.8141652613827993
Best split (array([     0,      1,      2, ..., 109726, 109727, 109728]), array([109729, 109730, 109731, ..., 112691, 112692, 112693]))


c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### TODO: 可試著使用原本的方法(pred_proba)，先找到閾值再進行判斷

In [76]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_MacroJur = splits_MacroJur[-1][0]
test_idx_MacroJur = splits_MacroJur[-1][1]

In [77]:
# Extract test and train data from TheWhole_rev_jur1 data
X_train_MacroJur, X_test_MacroJur = X_Important20Jur.iloc[train_idx_MacroJur], X_Important20Jur.iloc[test_idx_MacroJur]
y_train_MacroJur, y_test_MacroJur = y_Important20Jur.iloc[train_idx_MacroJur], y_Important20Jur.iloc[test_idx_MacroJur]

#### Data Standardization

In [78]:
# 初始化 StandardScaler
scaler = StandardScaler()

# 擬合並轉換訓練數據
X_train_MacroJur = scaler.fit_transform(X_train_MacroJur)

# 使用相同的scaler來轉換測試數據
X_test_MacroJur = scaler.fit_transform(X_test_MacroJur)

#### Feature Selection

In [79]:
# 初始化 Logistic Regression 模型
model = LogisticRegression()

# 初始化 RFE，並設定要選擇的特徵數量
selector_MacroJur = RFE(estimator=model, n_features_to_select=15)

# 在標準化後的訓練數據上擬合 RFE
selector_MacroJur = selector_MacroJur.fit(X_train_MacroJur, y_train_MacroJur)

# 獲取被選擇的特徵
selected_features_MacroJur = X_Important20Jur.columns[selector_MacroJur.support_]

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

In [80]:
print(selected_features_MacroJur)

Index(['revd1', 'revd12', 'revd2', 'revd6', 'revd7', 'revd11', 'revd5',
       'NominalGDP', 'FX', '貨幣總額年增率(M1A)日平均', '台灣外銷訂單金額–全球',
       '台灣外銷訂單金額–資訊與通信產品', '進口物價指數年增率', '台灣景氣同時指標', '台灣景氣落後指標'],
      dtype='object')


#### Model Training

In [81]:
# 使用選擇的特徵來訓練模型
model.fit(X_train_MacroJur[:, selector_MacroJur.support_], y_train_MacroJur)

LogisticRegression()

In [82]:
# 使用模型來預測測試數據
y_pred_MacroJur = model.predict(X_test_MacroJur[:, selector_MacroJur.support_])

#### Accuracy

In [83]:
# 計算精準度
accuracy_MacroJur = accuracy_score(y_test_MacroJur, y_pred_MacroJur)

print("Model accuracy: ", round(accuracy_MacroJur, 3))

Model accuracy:  0.637


### 加入三大法人買賣超訓練

#### Data SCleaning

In [84]:
# Add 三大法人 data to TheWhole_rev_jur1
TheWhole_rev_jur2 = pd.merge(TheWhole_rev_jur1, JuridicalInfo_last_day, how='inner', on=('ddate', 'stockid'))
display(TheWhole_rev_jur2.columns)

Index(['ddate', 'stockid', 'rev', 'mom', 'yoy', 'revd1', 'yoyd1', 'mond1',
       'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4', 'yoyd4',
       'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6', 'revd7',
       'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9', 'mond9',
       'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11', 'revd12',
       'yoyd12', 'mond12', 'FX', 'category_FX', 'CCI', 'category_CCI',
       'CPI_growth', 'category_CPIgrowth', 'NominalGDP', 'GDP_growth',
       'category_GDPgrowth', 'direction', 'ActualDirect',
       'GrandTransAmount(K)_x', 'monthly_ShareRatio_change_x',
       '貨幣總額年增率(M1A)日平均', '貨幣總額年增率(M2)日平均', '台灣外銷訂單金額–全球', '台灣外銷訂單金額–大陸及香港',
       '台灣外銷訂單金額–資訊與通信產品', '台灣外銷訂單年增率', '進口物價指數年增率', '台灣景氣領先指標', '台灣景氣同時指標',
       '台灣景氣落後指標', 'GrandTransAmount(K)_y', 'monthly_ShareRatio_change_y'],
      dtype='object')

In [85]:
X_Important15Jur2 = TheWhole_rev_jur2[['revd1', 'revd12', 'revd2', 'revd6', 'revd7', 'revd11', 'NominalGDP',
'FX', '貨幣總額年增率(M1A)日平均', '貨幣總額年增率(M2)日平均', '台灣外銷訂單金額–全球',
'台灣外銷訂單金額–資訊與通信產品', '進口物價指數年增率', '台灣景氣同時指標', '台灣景氣落後指標', 'GrandTransAmount(K)_y', 'monthly_ShareRatio_change_y']]

y_Important15Jur2 = TheWhole_rev_jur2['ActualDirect']

#### Data Preprocessing

In [86]:
# Split Data
# Split data by TimeSeriesSplit
tscv_MacroJur2 = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_MacroJur2 = list(tscv_MacroJur2.split(TheWhole_rev_jur2))

In [87]:
# Data Standardization

# 初始化 StandardScaler
scaler2 = StandardScaler()

# 擬合並轉換訓練數據
X_Important15Jur2 = scaler2.fit_transform(X_Important15Jur2)

In [101]:
#print(TheWhole_rev_jur2.info())
desc = TheWhole_rev_jur2.describe()

# 對標準差進行排序，並且選擇最大的三個索引
idx_std = np.argsort(desc.loc['std'])[-3:]

# 選擇標準差最大的三個欄位
desc.iloc[:, idx_std]

,revd3,revd4,rev
count,1.512390e+05,1.512390e+05,1.512390e+05
mean,1.810535e+06,1.808103e+06,1.822582e+06
std,1.342034e+07,1.342680e+07,1.348159e+07
min,-2.884635e+07,-2.884635e+07,-2.884635e+07
25%,8.243350e+04,8.247650e+04,8.231950e+04
50%,2.259320e+05,2.256730e+05,2.269140e+05
75%,6.687045e+05,6.675235e+05,6.731860e+05
max,8.223234e+08,8.223234e+08,8.223234e+08


In [103]:
TheWhole_rev_jur2.columns

Index(['ddate', 'stockid', 'rev', 'mom', 'yoy', 'revd1', 'yoyd1', 'mond1',
       'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4', 'yoyd4',
       'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6', 'revd7',
       'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9', 'mond9',
       'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11', 'revd12',
       'yoyd12', 'mond12', 'FX', 'category_FX', 'CCI', 'category_CCI',
       'CPI_growth', 'category_CPIgrowth', 'NominalGDP', 'GDP_growth',
       'category_GDPgrowth', 'direction', 'ActualDirect',
       'GrandTransAmount(K)_x', 'monthly_ShareRatio_change_x',
       '貨幣總額年增率(M1A)日平均', '貨幣總額年增率(M2)日平均', '台灣外銷訂單金額–全球', '台灣外銷訂單金額–大陸及香港',
       '台灣外銷訂單金額–資訊與通信產品', '台灣外銷訂單年增率', '進口物價指數年增率', '台灣景氣領先指標', '台灣景氣同時指標',
       '台灣景氣落後指標', 'GrandTransAmount(K)_y', 'monthly_ShareRatio_change_y'],
      dtype='object')

#### 使用所有分割組合進行交叉驗證

In [ ]:
# Create a model
model2 = LogisticRegression()

# 建立時間序列交叉驗證
tscv2 = TimeSeriesSplit(n_splits=50)

# 訓練模型並找出最佳的分割數據和模型參數
model.fit(X_Important15Jur2, y_Important15Jur2, )

# 取得最佳的分割數據的index
best_index2 = model2.best_index_

# 取得最佳的模型參數
best_estimator2 = model.best_estimator_

# 使用最佳的模型參數訓練整個數據集
best_estimator2.fit(X_Important15Jur2, y_Important15Jur2)

# 預測新的數據
y_pred2 = best_estimator2.predict(X_new)

print(y_pred2)

TypeError: float() argument must be a string or a real number, not 'TimeSeriesSplit'

Data Spliting 通常在資料預處理的最後一步

資料清洗、資料轉換(EX標準化)、資料集成